## Sample Module with Sample ModuleState

This notebook shows how to create a basic module that can be easily tested with the simulation

In [ ]:
import attr

from nlisim.module import Module, ModuleState
from nlisim.state import State


@attr.s(kw_only=True)
class SampleState(ModuleState):
    variable_name: int = attr.ib(default=0)

class Sample(Module):
    name = 'sample'
    defaults = {
        'ex_int': '0',
        'ex_float': '0.0',
        'ex_bool': 'False',
        'ex_string': '',
    }
    StateClass = SampleState
    
    def initialize(self, state: State):
        # how to get different values from config.ini file
        state.sample.ex_int = self.config.getint('ex_int')
        state.sample.ex_float = self.config.getfloat('ex_float')
        state.sample.ex_bool = self.config.getboolean('ex_bool')
        state.sample.ex_string = self.config.get('ex_string')
        return state
        
    # see nlisim.module to see various default functions whose behaviour can be overridden

In [ ]:
# see run in simulation/cli.py
from math import ceil

from nlisim.config import SimulationConfig
from nlisim.solver import run


# any config files in config_files should be hardcoded to be a full local file path
# Note: files outputted will be in the same folder as this notebook
config_files = []
config = SimulationConfig(*config_files)

config.add_module(Sample)

target_time = 5

state = run(config, target_time)